## **CAPSTONE PROJECT MODULE 2**

Crime in Boston

By Gregorius Wisangtitis Setyaji

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium as fo
from folium.plugins import HeatMap

## **IMPORTING DATA**


In [ ]:
df = pd.read_csv(r'D:\PURWADHIKA\CAPSTONE PROJECT\MODULE 2\Crime in Boston\crime.csv', encoding='windows-1254')

df.head()

## **Data Summary**

Data ini merupakan rekaman dari laporan tindak kriminal yang ada di Boston dari tanggal 14 Juni 2015 hingga 3 September 2018.
Terdapat 17 Kolom default, dengan penjabaran sebagai berikut:
1. INCIDENT_NUMBER = Unique Value, berupa nomor pelaporan tindak kriminal.
2. OFFENSE_CODE = Kode OFFENSE tindak kriminal 
3. OFFENSE_CODE_GROUP = Penamaan Grup dari OFFENSE_CODE / jenis tindak kriminal
4. OFFENSE_DESCRIPTION = Deskripsi dari OFFENSE_CODE_GROUP / tindak kriminal apa yang dilaporkan
5. DISTRICT = Lokasi distrik kejadian tindak kriminal yang dilaporkan
6. REPORTING_AREA = Klasifikasi kode distrik
7. SHOOTING = Apakah dalam kejadian tindak kriminal tersebut dilaporkan ada penembakan senjata api atau tidak
8. OCCURED_ON_DATE = Waktu kejadian tindak kriminal
9. YEAR = Tahun kejadian tindak kriminal
10. MONTH = Bulan kejadian tindak kriminal
11. DAY_OF_WEEK = Hari kejadian tindak kriminal
12. HOUR = Pada pukul berapa kejadian tindak kriminal
13. UCR_PART = Kategori tindak kriminal berdasarkan UCR
14. STREET = Di jalan apa kejadian tindak kriminal
15. LAT = Lokasi Latitude / lintang kejadian tindak kriminal
16. LONG = Lokasi Longitude / bujur kejadian tindak kriminal
17. LOCATION = Berisi Lat dan Long yang dijadikan satu string (tidak dipisah Lat dan Long)

Dapat dikatakan bahwa semua kolom yang ada pada data ini memiliki jenis data nominal (bukan numerik), tidak ada angka/value yang bisa dihitung dari semua kolom (pengecualian pada kolom Lat Long yang menandakan lokasi).

Berdasarkan keterangan di atas, dapat diasumsikan bahwa semua kolom memiliki distribusi yang tidak normal (data nominal kategorik, value dari setiap kolom hanya label dan tidak melambangkan kedudukan/tingkatan tertentu).

In [ ]:
df.info()

## **CHECK GENERAL INFO & DATA ANOMALIES**

Tahapan untuk check bagaimana bentuk datanya, apa tipe data tiap kolom, serta cek ada berapa baris data yang kosong (null data) tiap kolomnya.


In [ ]:
df.shape

In [ ]:
df.describe(include= 'all')

In [ ]:
print('\n','-'*30, "INFO TIAP KOLOM PADA DATA",'-'*30, '\n')
display(df.info())
print('-'*100)
print('\n','-'*30, "JUMLAH NULL DATA TIAP KOLOM",'-'*30, '\n')
display(df.isna().sum())

## **Handling Anomalies**

Tahap untuk menghilangkan data anomali.

- Pada kolom OCCURRED_ON_DATE seharusnya bertipe datetime.

- Sebanyak 0,5% baris pada kolom DISTRICT berupa null data, baris ini akan dihapus mengingat jumlah 0,5% dapat diasumsikan tidak akan merubah keseluruhan struktur data jika digunakan sebagai bahan analisis.

- Sebanyak 3% baris pada kolom STREET berupa null data, baris ini akan dihapus mengingat jumlah 3% dapat diasumsikan tidak akan merubah keseluruhan struktur data jika digunakan sebagai bahan analisis.

- Sebanyak 6% baris pada kolom Latitude dan Longtitude berupa null data. Namun terdapat kolom Location yang juga berisi informasi Decimal Degree Latitude dan Longtitude, sehingga kolom Latitude dan Longtitude akan digantikan datanya berdasarkan split data dari kolom Location.
Juga terdapat 3% row dari lat long yang memiliki value 0 dan -1, datanya akan di drop.

- Sebanyak 99% baris pada kolom SHOOTING berupa null data, sebanyak 99% null data ini dapat diartikan sebagai "No Shooting" dan akan diisikan dengan N (No Shooting).

- Membuat kolom YearMonth baru yang berasal dari gabungan kolom YEAR dan MONTH untuk mempermudah analisis time-series (tipe datetime).


In [ ]:
# Code untuk merubah tipe data

df['OCCURRED_ON_DATE'] = pd.to_datetime(df['OCCURRED_ON_DATE'])
# df['YEAR'] = df['YEAR'].apply(str)
# df['MONTH'] = df['MONTH'].apply(str)
# df['HOUR'] = df['HOUR'].apply(str)

In [ ]:
# Code untuk drop kolom yang tidak terpaikai (Lat Long akan diisikan dengan Location)

df.drop(['Lat', 'Long'], axis = 1, inplace=True)


In [ ]:
# Code untuk split kolom Location menjadi Lat dan Long

split = df['Location'].str.strip(')').str.strip('(').str.split(', ')
df['Lat'] = split.apply(lambda x: x[0])
df['Long'] = split.apply(lambda x: x[1])
df['Lat'] = pd.to_numeric(df['Lat'])
df['Long'] = pd.to_numeric(df['Long'])

In [ ]:
# value lat long 0 dan -1 menjadi nan

df['Lat'].replace(-1,np.nan,inplace=True)
df['Long'].replace(-1,np.nan,inplace=True)
df['Lat'].replace(0,np.nan,inplace=True)
df['Long'].replace(0,np.nan,inplace=True)

In [ ]:
df['SHOOTING'].replace(np.nan,'N',inplace=True)

In [ ]:
# Code untuk drop row yang kosong

df.dropna(subset=['DISTRICT'], inplace=True)
df.dropna(subset=['STREET'], inplace=True)
df.dropna(subset=['UCR_PART'], inplace=True)
df.dropna(subset=['Lat'], inplace=True)
df.dropna(subset=['Long'], inplace=True)

In [ ]:
df["MonthYear"] = df['YEAR'].astype(str) +"-"+ df["MONTH"].astype(str)
df['MonthYear'] = pd.to_datetime(df['MonthYear'])

In [ ]:
df.head()

## **Check Data Duplicate**

Terdapat data duplikat, akan ditangani dengan dihapus datanya.

In [ ]:
# Check Dupliacate

print(f'Jumlah data duplikat: ',df.duplicated().sum())
print('-'*100)
display(df[df.duplicated()])


In [ ]:
# Code untuk drop data duplikat

df.drop_duplicates(inplace=True)

## **Recheck Data**

In [ ]:
df.describe(include= 'all')

Terdapat INCIDENT_NUMBER ganda, namun keterangan pada kolom lainnya berbeda, seperti contoh memiliki OFFENSE_CODE_GROUP yang berbeda.

In [ ]:
df[df['INCIDENT_NUMBER'].duplicated()]

In [ ]:
print('\n','-'*30, "INFO TIAP KOLOM PADA DATA",'-'*30, '\n')
display(df.info())
print('-'*100)
print('\n','-'*30, "JUMLAH NULL DATA TIAP KOLOM",'-'*30, '\n')
display(df.isna().sum())

In [ ]:
# Check Dupliacate

print(f'Jumlah data duplikat: ',df.duplicated().sum())
print('-'*100)
display(df[df.duplicated()])

## **Merging Tables**

Terdapat data penjabaran dari OFFENSE_CODE dari tabel offense_codes.csv, akan diimport sebagai df2 dan nantinya akan digabungkan ke dataframe utama (df)
Kolom CODE pada Dataframe df2 terdapat duplikat yang memiliki NAME serupa namun penulisannya terdapat tambahan 'spasi', nantinya row duplicate ini akan didrop sebagai tahapan cleaning data df2. 

In [ ]:
# Import df2

df2 = pd.read_csv(r'D:\PURWADHIKA\CAPSTONE PROJECT\MODULE 2\Crime in Boston\offense_codes.csv', encoding='windows-1254')

df2.head()

In [ ]:
#  Check duplicate on CODE column

df2[df2['CODE'].duplicated()]

In [ ]:
# Contoh data duplikat

df2.loc[df2['CODE']==111]

In [ ]:
# Drop duplicate dari df2 kolom CODE

df2.drop_duplicates(subset = 'CODE', inplace=True)

In [ ]:
# Recheck Duplicate

df2[df2['CODE'].duplicated()]

In [ ]:
# Join df dan df2

df = pd.merge(df, df2, left_on='OFFENSE_CODE', right_on='CODE', how= 'left')

# Drop kolom yang tidak perlu ([CODE])

df.drop(['CODE'], axis= 1, inplace= True)

df.rename(columns = {'NAME':'OFFENSE_NAME'}, inplace = True)
df.insert(4, 'OFFENSE_NAME', df.pop('OFFENSE_NAME'))

df.head()

In [ ]:
# Check jumlah left table, sudah sama dengan sebelum join tables (296.399 rows)

df.info()

## **Big Question?**

1. Bagaimana tren jumlah pelaporan tindak kriminalitas di Boston dari tahun-ke-tahun?

2. Tindak kriminalitas apa yang paling sering dilaporkan di Boston? (annual dan total populasi data). OFFENSE_CODE_GROUP atau OFFENSE_DESCRIPTION.

3. Distrik mana yang banyak dilaporkan mengalami tindak kriminal? Apa jenisnya? DISTRICT, OFFENSE_CODE_GROUP.

4. Pelaporan tindak kriminalitas terbanyak berdasarkan jenis dan lokasi lat-long atau distrik. LAT-LONG or DISTRICT.

5. Adakah perpindahan kecenderungan lokasi tindak kriminalitas yang dilaporkan dari tahun-ke-tahun? OCCURRED_ON_DATE.

6. Adakah perbedaan jenis kriminalitas yang dilaporkan berdasarkan waktu dalam sehari? dengan anggapan:
    - 5-10 adalah pagi,
    - 11-16 adalah siang,
    - 17-22 adalah sore,
    - 23-4 adalah malam.

7. Adakah kecenderungan tindak kriminalitas tertentu yang dilaporkan di hari tertentu? OFFENSE_CODE_GROUP dan DAY_OF_WEEK.

## **Explore the Data**

Deskripsi statistik singkat dari data, dapat terlihat hal sebagai berikut:
- Dari 63 OFFENSE_CODE_GROUP / jenis tindak kriminal yang ada, paling sering dilaporkan adalah Motor Vehicle Accident Response (kecelakaan kendaraan bermotor).
- Terdapat 12 distrik di Boston, dan yang paling sering dilaporkan ada kejadian tindak kriminal adalah distrik B2.
- 99% dari total pelaporan kejadian tindak kriminal tidak disertai dengan penembakan senjata api (shooting).
- Pelaporan tindak kriminal paling sering dilaporkan di hari Jumat (Friday).
- Berdasarkan kategori UCR nya, paling sering dilaporkan adalah tindak kriminal dengan UCR Part Three.
- Tindak kriminal yang paling sering dilaporkan terjadi di jalan Washington.

In [ ]:
df.describe(include='all')

In [ ]:
df.head()

In [ ]:
print('Total semua Data (bersih) : ', df.shape[0])
print('Nilai Unik pada kolom INCIDENT_NUMBER : ', df['INCIDENT_NUMBER'].nunique())
print('\nTerdapat INCIDENT_NUMBER yang sama di dalam dataframe, namun setiap barisnya tetap unik karena OFFENSE_CODE yang berbeda.')

## **Tren Jumlah Tindak Kriminal di Boston Annually**

Jumlah data yang didapatkan adalah rekaman data dari Tahun 2015 sampai dengan 2018, namun perekaman data pada tahun 2015 dan 2018 memang tidak penuh selama satu tahun. Hal ini akan mengakibatkan missinterpretasi jika digunakan untuk melihat tren (mengingat jumlah data di tahun 2015 dan 2018 tidak sebanding dengan data 2016 dan 2017 yang durasi perekaman datanya satu tahun penuh).

In [ ]:
trendYear = df.groupby('YEAR')['INCIDENT_NUMBER'].count()

In [ ]:
sns.lineplot(data = trendYear)
plt.title('Grafik Jumlah Laporan Tindak Kriminal tahun 2015 - 2018')
plt.xticks([2015,2016,2017,2018])
plt.grid()


Jika dilihat pada grafik, terjadi kenaikan jumlah insiden tindak kriminal dari tahun 2016 ke 2017.
Grafik ini tidak bisa digunakan sebagai acuan, mengingat perekaman data pada tahun 2015 dan 2018 tidak berdurasi satu tahun penuh.

In [ ]:
strip = df[(df['MonthYear'] > '2015-07-01') & (df['MonthYear'] < '2018-09-01')]
trendMonth = strip.groupby('MonthYear')['INCIDENT_NUMBER'].count()


In [ ]:
fig,ax = plt.subplots(figsize=(15,15))

sns.lineplot(data = trendMonth, ax=ax)


plt.xticks(rotation = 90)
plt.xlabel("Tanggal", fontsize = 15)
plt.ylabel("Jumlah Tindak Kriminal", fontsize = 15)
plt.title("Tren Jumlah Tindak Kriminal dari Waktu ke Waktu", fontsize = 30)
plt.grid(alpha = 0.2)

# axvline untuk pembagian tahun
plt.axvline(pd.Timestamp('2015-12-31'), color='g', alpha = 0.9)
plt.axvline(pd.Timestamp('2016-12-31'), color='g', alpha = 0.9)
plt.axvline(pd.Timestamp('2017-12-31'), color='g', alpha = 0.9)

# axvline label pembagian tahun
plt.text(pd.Timestamp('2015-08-28'), y = 8400, s = '2015',color='g',ha = 'right', va = 'top', fontsize = 14)
plt.text(pd.Timestamp('2016-06-28'), y = 8400, s = '2016',color='g',ha = 'right', va = 'top', fontsize = 14)
plt.text(pd.Timestamp('2017-06-28'), y = 8400, s = '2017',color='g',ha = 'right', va = 'top', fontsize = 14)
plt.text(pd.Timestamp('2018-06-28'), y = 8400, s = '2018',color='g',ha = 'right', va = 'top', fontsize = 14)

# axvline untuk pembagian season
plt.axvline(pd.Timestamp('2015-09-23'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2015-12-22'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2016-03-21'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2016-06-21'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2016-09-23'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2016-12-22'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2017-03-21'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2017-06-21'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2017-09-23'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2017-12-22'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2018-03-21'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2018-06-21'), ls = '--', color='r', alpha = 0.3)
plt.axvline(pd.Timestamp('2018-09-23'), ls = '--', color='r', alpha = 0.3)

# axvline label season
plt.text(pd.Timestamp('2015-08-23'), y = 8000, s = 'Fall',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2015-11-22'), y = 8000, s = 'Winter',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2016-02-21'), y = 8000, s = 'Spring',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2016-05-21'), y = 8000, s = 'Summer',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2016-08-23'), y = 8000, s = 'Fall',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2016-11-22'), y = 8000, s = 'Winter',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2017-02-21'), y = 8000, s = 'Spring',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2017-05-21'), y = 8000, s = 'Summer',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2017-08-23'), y = 8000, s = 'Fall',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2017-11-22'), y = 8000, s = 'Winter',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2018-02-21'), y = 8000, s = 'Spring',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2018-05-21'), y = 8000, s = 'Summer',color='r',ha = 'right', va = 'top')
plt.text(pd.Timestamp('2018-08-23'), y = 8000, s = 'Fall',color='r',ha = 'right', va = 'top')

plt.show()

- Berdasarkan data dalam grafik tren bulanan, tren dari jumlah tindak kriminal memang sangat fluktuatif naik turun.

- Menariknya, jika dilihat berdasarkan 4 musim yang ada, pada musim semi (spring, sekitar pertengahan bulan Maret hingga Juni) jumlah tindak kriminal selalu paling rendah dibandingkan pada musim-musim yang lain.

- Selalu terjadi kenaikan jumlah tindak kriminal dari musim semi (spring) dan puncak pada tahun tersebut ada di musim panas (summer), kejadian ini selalu terjadi di tahun 2016 hingga 2018.

- Terlihat ada turning point penurunan jumlah laporan tindak kriminal yang selalu terjadi pada pergantian tahun (lihat garis hijau).

**Jenis Tindak Kriminalitas Terbanyak**

In [ ]:
df['OFFENSE_CODE_GROUP'].value_counts().nlargest(4)

In [ ]:
sns.catplot(y='OFFENSE_CODE_GROUP',
            kind='count',
            height=10,
            aspect=2,
            order=df['OFFENSE_CODE_GROUP'].value_counts().nlargest(4).index,
            data=df)
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Jumlah Tindak Kriminal", fontsize = 15)
plt.ylabel("Jenis Tindak Kriminal", fontsize = 15)
plt.title("Jumlah Tindak Kriminal berdasarkan Jenisnya", fontsize = 30)

Berdasarkan grafik di atas, tindak kriminal terbanyak adalah Motor Vehicle Accident Response

## **Tindak Kriminal Terbanyak berdasarkan Distrik**

Dapat dilihat bahwa 5 distrik teratas yang memiliki laporan tindak kriminal terbanyak adalah distrik:
- D4
- B2
- C11
- A1
- B3

Sedangkan jenis tindak kriminal yang dilaporkan terbanyak berdasarkan distrik ini adalah Larceny dan Motor Vehicle Accident Response.

In [ ]:
df.groupby(['DISTRICT'])['OFFENSE_CODE_GROUP'].value_counts().nlargest(5)

In [ ]:
# Map the Top District

topdist = df[df['DISTRICT'].isin(['D4', 'B2', 'C11', 'A1', 'B3'])]

fig = px.scatter_mapbox(topdist,
                        center = dict(lat = 42.321145,                       
                        lon = -71.057083),
                        zoom = 10,
                        lat='Lat', lon='Long',
                        hover_name='INCIDENT_NUMBER',
                        hover_data=['DISTRICT', 'OFFENSE_CODE_GROUP'],
                        height=1200,
                        color='DISTRICT')

fig.update_layout(mapbox_style='open-street-map')
fig.show()

## **Tindak Kriminal Berdasarkan Lokasi**

Berdasarkan 4 Jenis Tindak Kriminal terbanyak yang dilaporkan dari durasi perekaman data:
- Motor Vehicle Accident Response cenderung banyak dilaporkan di distrik A1, B2, B3, C11, dan D4 ;
- Larceny cenderung banyak dilaporkan di distrik A1 dan D4;
- Medical Assistance banyak dilaporkan di distrik D4, B2, dan C11;
- Investigate Person banyak dilaporkan di distrik D4, B2, dan C11.

Distrik yang memiliki kecenderungan hotspot laporan tindak kriminal memiliki kesamaan, yaitu dekat dengan perairan, yang mana bisa diartikan sebagai area yang dekat dengan pelabuhan sebagai lokasi aktivitas ekonomi yang cukup padat.

In [ ]:
# Buat Heatmap All Data

map_heat = fo.Map(location=[42.321145,-71.057083],
                    zoom_start = 11,
                    min_zoom=10,
                    tiles= 'openstreetmap' )

fo.CircleMarker([42.331145,-71.064083],
                    radius=70,
                    color='red',
                    ).add_to(map_heat)

data_heatmap = df[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=15).add_to(map_heat)

map_heat

**OFFENSE CODE GROUP = Motor Vehicle Accident Response (MVAR)**

In [ ]:
# Filter data hanya M V A R
MVAR = df[df['OFFENSE_CODE_GROUP']=='Motor Vehicle Accident Response']

# Buat Heatmapnya
map_MVAR = fo.Map(location=[42.326145,-71.057083],
                    zoom_start = 11,
                    min_zoom=10,
                    tiles= 'openstreetmap' )

fo.CircleMarker([42.320145,-71.064083],
                    radius=70,
                    color='red',
                    ).add_to(map_MVAR)

data_heatmap = MVAR[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=14).add_to(map_MVAR)

map_MVAR

**OFFENSE CODE GROUP = Larency**

In [ ]:
# Filter data hanya Larceny
LAR = df[df['OFFENSE_CODE_GROUP']=='Larceny']

# Buat Heatmapnya
map_LAR = fo.Map(location=[42.341145,-71.057083],
                    zoom_start = 11,
                    min_zoom=10,
                    tiles= 'openstreetmap' )

fo.CircleMarker([42.341145,-71.064083],
                    radius=50,
                    color='red',
                    ).add_to(map_LAR)

data_heatmap = LAR[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=13).add_to(map_LAR)

map_LAR

**OFFENSE CODE GROUP = Medical Assistance**

In [ ]:
# Filter data hanya Medical Assistance
MA = df[df['OFFENSE_CODE_GROUP']=='Medical Assistance']

# Buat Heatmapnya
map_MA = fo.Map(location=[42.321145,-71.057083],
                    zoom_start = 11,
                    min_zoom=10,
                    tiles= 'openstreetmap' )

fo.CircleMarker([42.331145,-71.064083],
                    radius=70,
                    color='red',
                    ).add_to(map_MA)

data_heatmap = MA[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=14).add_to(map_MA)

map_MA

**OFFENSE CODE GROUP = Investigate Person**

Berdasarkan plot heatmap, tindak kejahatan Investigate Person kerap terjadi di

In [ ]:
# Filter data hanya Investigate Person
IP = df[df['OFFENSE_CODE_GROUP']=='Investigate Person']

# Buat Heatmapnya
map_IP = fo.Map(location=[42.301145,-71.057083],
                    zoom_start = 11, min_zoom=11, tiles= 'openstreetmap')

fo.CircleMarker([42.321145,-71.064083],
                    radius=100,
                    color='red',
                    ).add_to(map_IP)

data_heatmap = IP[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=14).add_to(map_IP)

map_IP

## **Kecenderungan Perpindahan Tindak Kriminal dari Tahun ke Tahun**

Berdasarkan plot heatmap ke peta, kecenderungan hotspot laporan tindak kriminal setiap tahunnya berada di distrik A1, B4, C11, dan D2.
Distrik yang memiliki kecenderungan hotspot laporan tindak kriminal setiap tahunnya memiliki kesamaan, yaitu dekat dengan perairan, yang mana bisa diartikan sebagai area yang dekat dengan pelabuhan sebagai lokasi aktivitas ekonomi yang cukup padat.

**Tahun 2015 (Heatmap All)**

In [ ]:
# Filter data hanya tahun 2015
y15 = df[df['YEAR']== 2015]

# Buat Heatmapnya
map_y15 = fo.Map(location=[42.321145,-71.057083],
                    zoom_start = 11,
                    min_zoom=11,
                    tiles= 'openstreetmap' )

fo.CircleMarker([42.331145,-71.064083],
                    radius=70,
                    color='red',
                    ).add_to(map_y15)

data_heatmap = y15[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=14).add_to(map_y15)

map_y15

**Tahun 2016 (Heatmap All)**

In [ ]:
# Filter data hanya tahun 2016
y16 = df[df['YEAR']== 2016]

# Buat Heatmapnya
map_y16 = fo.Map(location=[42.321145,-71.057083],
                    zoom_start = 11,
                    min_zoom=11,
                    tiles= 'openstreetmap' )

fo.CircleMarker([42.331145,-71.064083],
                    radius=70,
                    color='red',
                    ).add_to(map_y16)

data_heatmap = y16[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=14).add_to(map_y16)

map_y16

**Tahun 2017 (Heatmap All)**

In [ ]:
# Filter data hanya tahun 2017
y17 = df[df['YEAR']== 2017]

# Buat Heatmapnya
map_y17 = fo.Map(location=[42.321145,-71.057083],
                    zoom_start = 11,
                    min_zoom=11,
                    tiles= 'openstreetmap' )

fo.CircleMarker([42.331145,-71.064083],
                    radius=70,
                    color='red',
                    ).add_to(map_y17)

data_heatmap = y17[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=14).add_to(map_y17)

map_y17

**Tahun 2018 (Heatmap All)**

In [ ]:
# Filter data hanya tahun 2018
y18 = df[df['YEAR']== 2018]

# Buat Heatmapnya
map_y18 = fo.Map(location=[42.321145,-71.057083],
                    zoom_start = 11,
                    min_zoom=11,
                    tiles= 'openstreetmap' )

fo.CircleMarker([42.331145,-71.064083],
                    radius=70,
                    color='red',
                    ).add_to(map_y18)

data_heatmap = y18[['Lat','Long']]
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=14).add_to(map_y18)

map_y18

## **Perbedaan Jumlah Tindak Kriminal berdasarkan Waktu dalam sehari**

Tindak kriminal yang dilaporkan cenderung banyak di pagi dan siang hari.
Berdasarkan waktunya, jenis tindak kriminal tertinggi sebagai berikut:

    - Malam hari adalah Motor Vehicle Accident Response sebanyak 4331
    - Pagi hari adalah Motor Vehicle Accident Response sebanyak 6629
    - Siang hari adalah Larceny sebanyak 10633
    - Sore hari adalah Motor Vehicle Accident Response sebanyak 9605


In [ ]:
# Penambahan kolom pagi-siang-sore-malam

def cat(row):
    if 5 <= row['HOUR'] <= 10:
        val = 'Pagi'
    elif 11 <= row['HOUR'] <= 16:
        val = 'Siang'
    elif 17 <= row['HOUR'] <= 22:
        val = 'Sore'
    else:
        val = 'Malam'
    return val

df['HOUR_CAT'] = df.apply(cat, axis = 1)

In [ ]:
df.groupby('HOUR_CAT')['OFFENSE_CODE_GROUP'].value_counts().nlargest(20).sort_index()

In [ ]:
# Grafik jumlah tindakan kriminal berdasarkan kategori waktu (HOUR_CAT)

sns.catplot(x='HOUR_CAT',
            kind='count',
            height=15, 
            aspect=1,
            data=df,
            order=['Pagi', 'Siang', 'Sore', 'Malam'])
plt.xticks(size=30)
plt.yticks(size=30)
plt.title('Grafik Jumlah Tindak Kriminal Berdasarkan Kategori Waktu', fontsize = 40)
plt.ylabel('Jumlah Tindak Kriminal', fontsize=30)

## **Perbedaan Jumlah Tindak Kriminal berdasarkan Hari**

Jumlah pelaporan tindak kriminal terbanyak ada di hari Jumat (Friday), dengan persentase 15,22%.

In [ ]:
sns.catplot(x = 'DAY_OF_WEEK',
           kind = 'count',
           height = 15, 
           aspect = 2,
           data = df,
           order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'] )
plt.xticks(size = 30)
plt.yticks(size = 30)
plt.xlabel('')
plt.ylabel('Count', fontsize = 40)

In [ ]:
dow_count = df['DAY_OF_WEEK'].value_counts()

plt.pie(data = df,
        x = dow_count,
        labels = ['Friday', 'Wednesday', 'Thursday', 'Tuesday', 'Monday', 'Saturday',
       'Sunday'],
        colors = sns.color_palette('bright'),
        autopct = '%.2f%%'
           )
plt.xticks(size = 30)
plt.yticks(size = 30)



## **Kecenderungan Jenis Tindak Kriminal di Hari Tertentu**

Pelaporan Motor Vehicle Accident Response adalah jenis terbanyak di setiap harinya, diikuti dengan Larceny dan Medical Assistance. 

In [ ]:
df.groupby(['DAY_OF_WEEK'])['OFFENSE_CODE_GROUP'].value_counts().nlargest(14).sort_index()

## **Conclusions**

Keismpulan dari analisis sebagai berikut:
1. - Jumlah pelaporan tindak kriminal mengalami peningkatan pada tahun 2016 ke 2017.
   - Berdasarkan 4 musim yang ada, pada musim semi (spring, sekitar pertengahan bulan Maret hingga Juni) jumlah tindak kriminal selalu paling rendah dibandingkan pada musim-musim yang lain.
   - Selalu terjadi kenaikan jumlah tindak kriminal dari musim semi (spring) dan puncak pada tahun tersebut ada di musim panas (summer), kejadian ini selalu terjadi di tahun 2016 hingga 2018.

2. Jenis tindak kriminal yang dilaporkan terbanyak sebagai berikut:
    - Motor Vehicle Accident Response sebanyak   30376
    - Larceny                             sebanyak 25057
    - Medical Assistance                  sebanyak 22361
    - Investigate Person                  sebanyak 17963

3. Distrik yang banyak dilaporkan terdapat tindak kriminal dan juga jenisnya sebagai berikut:

    - Pada Distrik D4        adalah Larceny sebanyak                            7141
    - Pada Distrik B2        adalah Motor Vehicle Accident Response sebanyak   4971
    - Pada Distrik C11      adalah Motor Vehicle Accident Response sebanyak    4758
    - Pada Distrik A1       adalah Larceny sebanyak                            4477
    - Pada Distrik B3        adalah Motor Vehicle Accident Response sebanyak    3255

4. Berdasarkan 4 Jenis Tindak Kriminal terbanyak yang dilaporkan dari durasi perekaman data:
    - Motor Vehicle Accident Response cenderung banyak dilaporkan di distrik A1, B2, B3, C11, dan D4 ;
    - Larceny cenderung banyak dilaporkan di distrik A1 dan D4;
    - Medical Assistance banyak dilaporkan di distrik B2, C11, dan D4;
    - Investigate Person banyak dilaporkan di distrik B2, C11, dan D4.

5. Kecenderungan kejadian tindak kriminal yang dilaporkan ada di distrik A1, B4, C11, dan D12.
Distrik yang memiliki kecenderungan hotspot laporan tindak kriminal setiap tahunnya memiliki kesamaan, yaitu dekat dengan perairan, yang mana bisa diartikan sebagai area yang dekat dengan pelabuhan sebagai lokasi aktivitas ekonomi yang cukup padat.

6. Tindak kriminal yang dilaporkan cenderung banyak di pagi dan siang hari.
Berdasarkan waktunya, jenis tindak kriminal tertinggi sebagai berikut:

    - Malam hari adalah    Motor Vehicle Accident Response     sebanyak 4331
    - Pagi hari adalah      Motor Vehicle Accident Response     sebanyak 6629
    - Siang hari adalah     Larceny                            sebanyak 10633
    - Sore hari adalah     Motor Vehicle Accident Response     sebanyak 9605

7. Pelaporan Motor Vehicle Accident Response adalah jenis terbanyak di setiap harinya, diikuti dengan Larceny dan Medical Assistance. 



## **Recommendation**

Berdasarkan hasil analisis data yang dimiliki (14 Juni 2015 hingga 3 September 2018), terdapat beberapa rekomendasi yang dapat diberikan, antara lain:

1. Jenis laporan terbanyak adalah kecelakaan kendaraan bermotor (Motor Vehicle Accident Response), bisa jadi penempatan rambu lalu lintas kurang baik atau jumlah rambu lalu lintas tidak memadai. Bisa menjadi dasar untuk kegiatan Monitoring dan Evaluasi pemasangan rambu lalu lintas. 
Jenis laporan terbanyak kedua adalah tindak pencurian (Larceny), hal ini mungkin dapat dimitigasi risikonya dengan menambahkan CCTV di sudut-sudu jalan.
Jenis laporan terbanyak ketiga adalah keperluan tindakan medis (Medical Assistance), dapat menjadi dasar dalam pemberian pelatihan pada petugas yang khususnya berpatroli di distrik B2, C11, dan D4 (banyak laporan keperluan tindakan medis di ketiga distrik tersebut).

2. Pelaporan kejadian kecelakaan kendaraan bermotor selalu dilaporkan di setiap hari, diikuti juga tindak pencurian dan kebutuhan tindakan medis.

3. Terdapat tren bahwa pada musim gugur (Fall), tindak kriminal yang dilaporkan jumlahnya paling banyak. Sebaliknya pada awal hingga pertengahan musim semi (Spring), tindak kriminal yang dilaporkan nilainya paling sedikit. Hal ini dapat menjadi dasar untuk evaluasi penambahan jumlah personil yang bertugas dalam jangka waktu tertentu (bisa jadi menggunakan skema penambahan shift kerja).

4. Distrik yang banyak mendapat laporan adalah distrik A1, B3, B4, C11, dan D2 yang mana lokasinya dekat dengan perairan/dermaga. Dapat menjadi dasar untuk menambahkan personil polisi atau CCTV untuk memitigasi banyaknya tindak kriminal. 

5. Jika terdapat kepentingan berinvestasi, rekreasi (turisme), atau mencari rumah tinggal, bisa menjadi pertimbangan untuk menghindari distrik dengan pelaporan terbanyak, yaitu menghindari distrik A1, B3, B4, C11, dan D2.
    

In [ ]:
df.to_csv(r'D:\PURWADHIKA\CAPSTONE PROJECT\MODULE 2\Crime in Boston\Crime_Boston_Clean.csv')